<a href="https://colab.research.google.com/github/subhrajit-mohanty/medium-blogs/blob/master/colab-notebooks/Finetune_SQL_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install oxenai torch pandas pyarrow fastparquet ipywidgets huggingface-hub einops trl bitsandbytes
!pip install git+https://github.com/huggingface/optimum.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install --upgrade accelerate

  Cloning https://github.com/huggingface/optimum.git to /tmp/pip-req-build-c093n0s3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/optimum.git /tmp/pip-req-build-c093n0s3
  Resolved https://github.com/huggingface/optimum.git to commit 4a7cb298140ee9bed968d98a780a950d15bb2935
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-0h0bipud
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-0h0bipud
  Resolved https://github.com/huggingface/transformers.git to commit 7f95372c6267d3163fd2aa74aeff9d84ddb6cc35
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch, oxen, json
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, TaskType, get_peft_model, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from torch.utils.data import Dataset, DataLoader

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
MODEL_NAME = "microsoft/phi-2" # "google/gemma-2-9b" #  microsoft/phi-2

In [6]:

# If there are issues with the tokenizer, uncomment the line below and comment out the other one
# tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, pad_token="[PAD]", trust_remote_code=True, use_fast=False)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True, use_fast=False)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

In [7]:

# Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# LoRA Config
lora_config = LoraConfig(
    r=16, # LoRA Rank (controls number of trainable parameters)
    lora_alpha=20, # LoRA Alpha (controls the size of the effect on the original model)
    target_modules="all-linear",
    lora_dropout=0.02,
    bias="none",
    task_type="CAUSAL_LM"
)

In [8]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, quantization_config=bnb_config, trust_remote_code=True, device_map="auto")
model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [9]:
model.train()
model.enable_input_require_grads()
model.print_trainable_parameters()

trainable params: 25,165,824 || all params: 3,846,245,376 || trainable%: 0.6543


In [10]:
# !rm -Rf 'Text2SQL'
# oxen.clone('greg/Text2SQL')

In [11]:
class SQLDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=2048):
        self.tokenizer = tokenizer
        self.data = []
        self.max_length = max_length

        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
              if i >= 50000000:
                break
              self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        record = self.data[idx]
        output = record["output"]
        query = record["query"]
        schema = record["schema"]

        input = f"Schema: {schema}\nInstructions: {query}\nAnswer: {output}"

        encoding = self.tokenizer(input, truncation=True, max_length=self.max_length, return_tensors="pt")

        return {'input_ids': encoding.input_ids.squeeze(0), 'attention_mask': encoding.attention_mask.squeeze(0)}

In [12]:
file_path = "Text2SQL/train.jsonl"
dataset = SQLDataset(file_path=file_path, tokenizer=tokenizer)

In [13]:
eval_file_path = "Text2SQL/test.jsonl"
eval_dataset = SQLDataset(file_path=eval_file_path, tokenizer=tokenizer)

In [14]:
instruction_template = "Schema:"
response_template = "Answer:"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:130: UserWarning: The pad_token_id and eos_token_id values of this tokenizer are identical. If you are planning for multi-turn training, it can result in the model continuously generating questions and answers without eos token. To avoid this, set the pad_token_id to a different value.
  warnings.warn(


In [15]:
!rm -Rf 'training'

In [16]:
training_args = TrainingArguments(
    output_dir="./training",
    warmup_steps=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_checkpointing=False, # True to save memory
    gradient_accumulation_steps=32,
    num_train_epochs=50,
    bf16=True,
    optim="paged_lion_8bit",
    learning_rate=1e-4,
    weight_decay=0.01,
    save_strategy="epoch", # Alternatively steps
    # save_steps=50, # If you're using steps
    # evaluation_strategy="steps", # To do evaluation
    # eval_steps=200 # To do evaluation
)


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=training_args,
    dataset_text_field="text",
    data_collator=collator,
    max_seq_length=2048
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This migh

In [17]:
import wandb
wandb.init(mode="disabled")

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:183: UserWarning: Could not find response key `Answer:` in the following instance: Schema: CREATE TABLE location (LocationID INTEGER, Country TEXT, State TEXT, StateCode TEXT, City TEXT);
CREATE TABLE user (UserID TEXT, Gender TEXT);
CREATE TABLE twitter (TweetID TEXT, Weekday TEXT, Hour INTEGER, Day INTEGER, Lang TEXT, IsReshare TEXT, Reach INTEGER, RetweetCount INTEGER, Likes INTEGER, Klout INTEGER, Sentiment REAL, text TEXT, LocationID INTEGER, UserID TEXT);

Instructions: How many tweets in total were posted by a user in Argentina?
Answer: SELECT COUNT(T1.TweetID) FROM twitter AS T1 INNER JOIN location AS T2 ON T2.LocationID = T1.LocationID WHERE T2.Country = 'Argentina' LIMIT 1; This instance will be ignored

OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 282.81 MiB is free. Process 347506 has 39.28 GiB memory in use. Of the allocated memory 38.31 GiB is allocated by PyTorch, and 480.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trained_model = PeftModel.from_pretrained(model, "training/checkpoint-20")

In [ ]:

def generate_text(prompt, model, max_new_tokens=2048):
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        output_ids = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id, do_sample=False)

    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text

In [ ]:
instruction = """Schema: CREATE TABLE People (Name TEXT, Age INTEGER, Height INTEGER, Weight INTEGER);
Instructions: Find all names with a length of 8 characters.
Answer:"""

print(generate_text(instruction, trained_model, 80))

In [ ]:
instruction = """Schema: CREATE TABLE People (Name TEXT, Age INTEGER, Height INTEGER, Weight INTEGER);
Instructions: Find all People who who's age is below 25 and height more then 170 cm and weight is less than 60
Answer:"""

print(generate_text(instruction, trained_model, 80))